# Proyek ReFisher

- **Nama:** [Nama Anda]
- **Dataset:** Fish Fresh and Non-Fresh Roboflow
- **Tujuan:** Membangun model klasifikasi gambar untuk membedakan ikan segar dan tidak segar.
- **Framework:** TensorFlow/Keras"

## 1. Import Libraries
Impor semua library yang dibutuhkan untuk pemrosesan data, visualisasi, dan persiapan model.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib

## 2. Unduh dan Ekstrak Dataset
Dataset diunduh dari Github Repo dan diekstrak ke environtment.